In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from collections import deque
from ipywidgets import interact
%matplotlib inline

In [2]:
class ProcessFrame:
    
    #def __init__(self,image):
    #    self.img = image
        
    def grayscale(self,image):
        return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    def canny(self,img, low_threshold, high_threshold):
        return cv2.Canny(img, low_threshold, high_threshold)

    def gaussian_blur(self,img, kernel_size):
        return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)
    
    def region_of_interest(self,img, vertices):
    #defining a blank mask to start with
        mask = np.zeros_like(img)
        if len(img.shape) > 2:
            channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
            ignore_mask_color = (255,) * channel_count
        else:
            ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
        cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
        masked_image = cv2.bitwise_and(img, mask)
        return masked_image

In [3]:
class DetectLane:
    def __init__(self,frame):
        #self.img = inp
        self.low_t = 50
        self.high_t = 150
        self.kernel = 3
        self.frame = frame
        self.angle_pairs = [(-50,-20),(20,50)]
        
    def calc_slope(self,lines):
        slopes = (lines[:,:,3] - lines[:,:,1])/(lines[:,:,2] - lines[:,:,0])
        return slopes
    
    def calc_theta(self,slopes):
        theta = np.arctan(slopes)
        theta_deg = np.degrees(theta)
        return theta_deg
    
    def segment_lane(self,lines,slope_degrees):
        leftlane = []
        rightlane = []
        for i,theta in enumerate(slope_degrees):
            if self.angle_pairs[0][0] < theta < self.angle_pairs[0][1]:
                leftlane.append(lines[i])
            if self.angle_pairs[1][0] < theta < self.angle_pairs[1][1] : 
                rightlane.append(lines[i])
        return leftlane,rightlane
    
    def decide_lane(self,lines):
        slopes = self.calc_slope(lines)
        slope_degrees = self.calc_theta(slopes)
        left_lane,right_lane = self.segment_lane(lines,slope_degrees)   
        return left_lane,right_lane
    
    def draw_extended(self,image,x,f,m,b):
        y1 = max(f(x))
        y2 = image.shape[1]
        x1 = (y1 - b)/m
        x2 = (y2 - b)/m
        cv2.line(image,(int(x1),int(y1)),(int(x2),int(y2)),(255,0,0),10)

    def draw_lines(self,image,lines,color = [255,0,0],thickness = 2):
        x= []
        y =[]
        lines = np.array(lines)
        x1 =lines[:,:,0].flatten()
        x2 =lines[:,:,2].flatten()
        y1 = lines[:,:,1].flatten()
        y2 = lines[:,:,3].flatten()
        for i in range(len(x1)):
            x +=[x1[i],x2[i]]
            y +=[y1[i],y2[i]]
        z = np.polyfit(x,y,1)
        m,b = z
        f = np.poly1d(z)
        cv2.line(image,(int(max(x)),int(f(max(x)))),(int(min(x)),int(f(min(x)))),(255,0,0),10)
        self.draw_extended(image,x,f,m,b)
        
    def process_image(self,image):
        
        vertices = np.array([[(50,image.shape[0]),(450,330),(550,330),(image.shape[1],image.shape[0])]],dtype = np.int32)
        line_image = np.copy(image) *0
        
        gray = self.frame.grayscale(image)
        blur = self.frame.gaussian_blur(gray,self.kernel)
        canny_img = self.frame.canny(blur,self.low_t,self.high_t)
        roi = self.frame.region_of_interest(canny_img,vertices)
        lines = cv2.HoughLinesP(roi,1,np.pi/120,20,np.array([]),15,40)
        left_lane,right_lane = self.decide_lane(lines)
        self.draw_lines(image,left_lane, color=[100, 0, 0], thickness=2)
        self.draw_lines(image,right_lane,color=[100,0,0],thickness = 2)
        lines_edges = cv2.addWeighted(image,0.8,line_image,0.2,0)
        result = lines_edges
        return result
          

In [4]:
def input_function():
    #image = mpimg.imread('test_images/solidWhiteRight.jpg')
    clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
    clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
    #print('This image is:', type(image), 'with dimensions:', image.shape)
    #plt.imshow(image)
    return clip1,clip2

In [5]:
if __name__ == "__main__":
    white_output = 'test_videos_output/solidWhiteRightclass.mp4'
    clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
    frame = ProcessFrame()
    detector = DetectLane(frame)
    white_clip = clip1.fl_image(detector.process_image)
    %time white_clip.write_videofile(white_output, audio=False)

t:   5%|▌         | 12/221 [00:00<00:01, 113.71it/s, now=None]

Moviepy - Building video test_videos_output/solidWhiteRightclass.mp4.
Moviepy - Writing video test_videos_output/solidWhiteRightclass.mp4



Moviepy - Done !
Moviepy - video ready test_videos_output/solidWhiteRightclass.mp4
CPU times: user 6.31 s, sys: 1.72 s, total: 8.03 s
Wall time: 3.82 s


In [6]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [7]:
yellow_output = 'test_videos_output/solidYellowLeftclass.mp4'
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
frame2 = ProcessFrame()
detector2 = DetectLane(frame2)
yellow_clip = clip2.fl_image(detector2.process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

t:   2%|▏         | 12/681 [00:00<00:05, 118.92it/s, now=None]

Moviepy - Building video test_videos_output/solidYellowLeftclass.mp4.
Moviepy - Writing video test_videos_output/solidYellowLeftclass.mp4



Moviepy - Done !
Moviepy - video ready test_videos_output/solidYellowLeftclass.mp4
CPU times: user 21.6 s, sys: 6.27 s, total: 27.9 s
Wall time: 12 s


In [8]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))